
# Advanced Recommender Systems with Python

Welcome to the code notebook for creating Advanced Recommender Systems with Python.

Recommendation Systems usually rely on larger data sets and specifically need to be organized in a particular fashion. Because of this, we won't have a project to go along with this topic, instead we will have a more intensive walkthrough process on creating a recommendation system with Python with the same Movie Lens Data Set.

*Note: The actual mathematics behind recommender systems is pretty heavy in Linear Algebra.*
___

## Methods Used

Two most common types of recommender systems are **Content-Based** and **Collaborative Filtering (CF)**. 

* Collaborative filtering produces recommendations based on the knowledge of users’ attitude to items, that is it uses the "wisdom of the crowd" to recommend items. 
* Content-based recommender systems focus on the attributes of the items and give you recommendations based on the similarity between them.

## Collaborative Filtering

In general, Collaborative filtering (CF) is more commonly used than content-based systems because it usually gives better results and is relatively easy to understand (from an overall implementation perspective). The algorithm has the ability to do feature learning on its own, which means that it can start to learn for itself what features to use. 

CF can be divided into **Memory-Based Collaborative Filtering** and **Model-Based Collaborative filtering**. 

In this tutorial, we will implement Model-Based CF by using singular value decomposition (SVD) and Memory-Based CF by computing cosine similarity. 

## The Data

We will use famous MovieLens dataset, which is one of the most common datasets used when implementing and testing recommender engines. It contains 100k movie ratings from 943 users and a selection of 1682 movies.

You can download the dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k.zip) or just use the u.data file that is already included in this folder.

____
## Getting Started

Let's import some libraries we will need:

In [1]:
import numpy as np
import pandas as pd

We can then read in the **u.data** file, which contains the full dataset. You can read a brief description of the dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt).

Note how we specify the separator argument for a Tab separated file.

In [2]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

Let's take a quick look at the data.

In [3]:
df.head()

user_id  item_id  rating  timestamp
0        0       50       5  881250949
1        0      172       5  881250949
2        0      133       1  881250949
3      196      242       3  881250949
4      186      302       3  891717742

Note how we only have the item_id, not the movie name. We can use the Movie_ID_Titles csv file to grab the movie names and merge it with this dataframe:

In [4]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

item_id              title
0        1   Toy Story (1995)
1        2   GoldenEye (1995)
2        3  Four Rooms (1995)
3        4  Get Shorty (1995)
4        5     Copycat (1995)

Then merge the dataframes:

In [5]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

user_id  item_id  rating  timestamp             title
0        0       50       5  881250949  Star Wars (1977)
1      290       50       5  880473582  Star Wars (1977)
2       79       50       4  891271545  Star Wars (1977)
3        2       50       5  888552084  Star Wars (1977)
4        8       50       5  879362124  Star Wars (1977)

Now let's take a quick look at the number of unique users and movies.

In [6]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+str(n_items))

Num. of Users: 944
Num of Movies: 1682


## Train Test Split

Recommendation Systems by their very nature are very difficult to evaluate, but we will still show you how to evaluate them in this tutorial. In order to do this, we'll split our data into two sets. However, we won't do our classic X_train,X_test,y_train,y_test split. Instead we can actually just segement the data into two sets of data:

In [7]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

## Memory-Based Collaborative Filtering

Memory-Based Collaborative Filtering approaches can be divided into two main sections: **user-item filtering** and **item-item filtering**. 

A *user-item filtering* will take a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked. 

In contrast, *item-item filtering* will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items and outputs other items as recommendations. 

* *Item-Item Collaborative Filtering*: “Users who liked this item also liked …”
* *User-Item Collaborative Filtering*: “Users who are similar to you also liked …”

In both cases, you create a user-item matrix which built from the entire dataset.

Since we have split the data into testing and training we will need to create two ``[943 x 1682]`` matrices (all users by all movies). 

The training matrix contains 75% of the ratings and the testing matrix contains 25% of the ratings.  

After you have built the user-item matrix you calculate the similarity and create a similarity matrix. 

The similarity values between items in *Item-Item Collaborative Filtering* are measured by observing all the users who have rated both items.  

For *User-Item Collaborative Filtering* the similarity values between users are measured by observing all the items that are rated by both users.


A distance metric commonly used in recommender systems is *cosine similarity*, where the ratings are seen as vectors in ``n``-dimensional space and the similarity is calculated based on the angle between these vectors. 
Cosine similiarity for users *a* and *m* can be calculated using the formula below, where you take dot product of  the user vector *$u_k$* and the user vector *$u_a$* and divide it by multiplication of the Euclidean lengths of the vectors.
<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

To calculate similarity between items *m* and *b* you use the formula:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(i_m,i_b)=\frac{i_m&space;\cdot&space;i_b&space;}{&space;\left&space;\|&space;i_m&space;\right&space;\|&space;\left&space;\|&space;i_b&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{a,m}x_{a,b}}{\sqrt{\sum&space;x_{a,m}^2\sum&space;x_{a,b}^2}}
"/>

Your first step will be to create the user-item matrix. Since you have both testing and training data you need to create two matrices.  

In [12]:
train_data.head()

user_id  item_id  rating  timestamp                       title
38814      592      343       3  882607476  Alien: Resurrection (1997)
39379      406      183       5  882480567                Alien (1979)
61398       43      820       2  884029742            Space Jam (1996)
81611      551       42       5  892783212               Clerks (1994)
13050      399        5       3  882345001              Copycat (1995)

In [13]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

Pandas(Index=23592, user_id=534, item_id=117, rating=5, timestamp=877807973, title='Rock, The (1996)')
Pandas(Index=31839, user_id=449, item_id=171, rating=4, timestamp=880410599, title='Delicatessen (1991)')
Pandas(Index=18843, user_id=430, item_id=151, rating=4, timestamp=877225516, title='Willy Wonka and the Chocolate Factory (1971)')
Pandas(Index=87219, user_id=201, item_id=123, rating=2, timestamp=884114233, title='Frighteners, The (1996)')
Pandas(Index=41753, user_id=524, item_id=709, rating=5, timestamp=884635171, title='Strictly Ballroom (1992)')
Pandas(Index=13367, user_id=538, item_id=237, rating=4, timestamp=877109986, title='Jerry Maguire (1996)')
Pandas(Index=87823, user_id=698, item_id=490, rating=3, timestamp=886366814, title='To Catch a Thief (1955)')
Pandas(Index=70276, user_id=60, item_id=523, rating=4, timestamp=883326837, title='Cool Hand Luke (1967)')
Pandas(Index=99743, user_id=179, item_id=1234, rating=1, timestamp=892151459, title='Chairman of the Board (1998)')

Pandas(Index=23803, user_id=128, item_id=117, rating=5, timestamp=879967631, title='Rock, The (1996)')
Pandas(Index=33258, user_id=74, item_id=7, rating=4, timestamp=888333458, title='Twelve Monkeys (1995)')
Pandas(Index=96429, user_id=788, item_id=503, rating=4, timestamp=880869984, title='Candidate, The (1972)')
Pandas(Index=82142, user_id=66, item_id=535, rating=4, timestamp=883602044, title='Addicted to Love (1997)')
Pandas(Index=42611, user_id=561, item_id=425, rating=4, timestamp=885808000, title='Bob Roberts (1992)')
Pandas(Index=22019, user_id=353, item_id=272, rating=5, timestamp=891402757, title='Good Will Hunting (1997)')
Pandas(Index=31629, user_id=405, item_id=554, rating=1, timestamp=885548049, title='Waterworld (1995)')
Pandas(Index=100, user_id=461, item_id=50, rating=3, timestamp=885356089, title='Star Wars (1977)')
Pandas(Index=50156, user_id=848, item_id=210, rating=5, timestamp=887039271, title='Indiana Jones and the Last Crusade (1989)')
Pandas(Index=82620, user_id

Pandas(Index=36626, user_id=407, item_id=705, rating=4, timestamp=875116117, title="Singin' in the Rain (1952)")
Pandas(Index=35364, user_id=653, item_id=367, rating=3, timestamp=878867228, title='Clueless (1995)')
Pandas(Index=4667, user_id=231, item_id=1, rating=3, timestamp=879965704, title='Toy Story (1995)')
Pandas(Index=6051, user_id=81, item_id=98, rating=5, timestamp=876534854, title='Silence of the Lambs, The (1991)')
Pandas(Index=776, user_id=16, item_id=172, rating=5, timestamp=877724726, title='Empire Strikes Back, The (1980)')
Pandas(Index=92193, user_id=782, item_id=894, rating=2, timestamp=891498031, title='Home Alone 3 (1997)')
Pandas(Index=23163, user_id=786, item_id=504, rating=4, timestamp=882843352, title='Bonnie and Clyde (1967)')
Pandas(Index=95180, user_id=790, item_id=1188, rating=3, timestamp=885157984, title='Young Guns II (1990)')
Pandas(Index=82809, user_id=694, item_id=528, rating=3, timestamp=875728842, title='Killing Fields, The (1984)')
Pandas(Index=7273

Pandas(Index=49975, user_id=301, item_id=210, rating=4, timestamp=882076211, title='Indiana Jones and the Last Crusade (1989)')
Pandas(Index=52753, user_id=537, item_id=732, rating=3, timestamp=886031912, title='Dave (1993)')
Pandas(Index=35683, user_id=892, item_id=11, rating=3, timestamp=886608897, title='Seven (Se7en) (1995)')
Pandas(Index=10180, user_id=28, item_id=143, rating=4, timestamp=881956564, title='Sound of Music, The (1965)')
Pandas(Index=14515, user_id=921, item_id=111, rating=4, timestamp=879380097, title='Truth About Cats & Dogs, The (1996)')
Pandas(Index=63895, user_id=423, item_id=269, rating=3, timestamp=891394558, title='Full Monty, The (1997)')
Pandas(Index=84056, user_id=130, item_id=356, rating=4, timestamp=880396792, title='Client, The (1994)')
Pandas(Index=88914, user_id=709, item_id=728, rating=4, timestamp=879849185, title='Junior (1994)')
Pandas(Index=58851, user_id=115, item_id=13, rating=5, timestamp=881171983, title='Mighty Aphrodite (1995)')
Pandas(Inde

Pandas(Index=61415, user_id=357, item_id=820, rating=4, timestamp=878952288, title='Space Jam (1996)')
Pandas(Index=81291, user_id=7, item_id=192, rating=4, timestamp=891352010, title='Raging Bull (1980)')
Pandas(Index=11, user_id=234, item_id=50, rating=4, timestamp=892079237, title='Star Wars (1977)')
Pandas(Index=77008, user_id=667, item_id=527, rating=4, timestamp=891035121, title='Gandhi (1982)')
Pandas(Index=96163, user_id=896, item_id=422, rating=3, timestamp=887159972, title='Aladdin and the King of Thieves (1996)')
Pandas(Index=93813, user_id=865, item_id=825, rating=1, timestamp=880144123, title='Arrival, The (1996)')
Pandas(Index=3139, user_id=44, item_id=222, rating=4, timestamp=883613334, title='Star Trek: First Contact (1996)')
Pandas(Index=95071, user_id=435, item_id=1225, rating=3, timestamp=884131597, title='Angus (1995)')
Pandas(Index=84502, user_id=435, item_id=101, rating=3, timestamp=884132184, title='Heavy Metal (1981)')
Pandas(Index=79349, user_id=342, item_id=27

...

Pandas(Index=46174, user_id=851, item_id=176, rating=4, timestamp=875731816, title='Aliens (1986)')
Pandas(Index=85466, user_id=234, item_id=650, rating=3, timestamp=892078837, title='Seventh Seal, The (Sjunde inseglet, Det) (1957)')
Pandas(Index=18005, user_id=239, item_id=174, rating=4, timestamp=889179131, title='Raiders of the Lost Ark (1981)')
Pandas(Index=10170, user_id=95, item_id=143, rating=4, timestamp=880571951, title='Sound of Music, The (1965)')
Pandas(Index=10539, user_id=533, item_id=423, rating=5, timestamp=888844906, title='E.T. the Extra-Terrestrial (1982)')
Pandas(Index=99573, user_id=655, item_id=1516, rating=3, timestamp=887474630, title='Wedding Gift, The (1994)')
Pandas(Index=95218, user_id=807, item_id=415, rating=3, timestamp=893082702, title='Apple Dumpling Gang, The (1975)')
Pandas(Index=31347, user_id=405, item_id=214, rating=4, timestamp=885545235, title='Pink Floyd - The Wall (1982)')
Pandas(Index=76808, user_id=648, item_id=411, rating=2, timestamp=882212

Pandas(Index=65404, user_id=159, item_id=9, rating=3, timestamp=880485766, title='Dead Man Walking (1995)')
Pandas(Index=17976, user_id=124, item_id=174, rating=3, timestamp=890287317, title='Raiders of the Lost Ark (1981)')
Pandas(Index=18821, user_id=382, item_id=151, rating=4, timestamp=875946830, title='Willy Wonka and the Chocolate Factory (1971)')
Pandas(Index=16915, user_id=862, item_id=208, rating=2, timestamp=879304282, title='Young Frankenstein (1974)')
Pandas(Index=12894, user_id=463, item_id=248, rating=3, timestamp=889935953, title='Grosse Pointe Blank (1997)')
Pandas(Index=19997, user_id=716, item_id=195, rating=1, timestamp=879795425, title='Terminator, The (1984)')
Pandas(Index=45993, user_id=184, item_id=176, rating=4, timestamp=889908740, title='Aliens (1986)')
Pandas(Index=27771, user_id=928, item_id=48, rating=5, timestamp=880936817, title='Hoop Dreams (1994)')
Pandas(Index=79967, user_id=450, item_id=1044, rating=4, timestamp=887139844, title='Paper, The (1994)')
P

Pandas(Index=22892, user_id=806, item_id=405, rating=3, timestamp=882385762, title='Mission: Impossible (1996)')
Pandas(Index=51936, user_id=665, item_id=185, rating=4, timestamp=884294200, title='Psycho (1960)')
Pandas(Index=31245, user_id=13, item_id=790, rating=2, timestamp=882141841, title='Tommy Boy (1995)')
Pandas(Index=51909, user_id=207, item_id=185, rating=4, timestamp=875509832, title='Psycho (1960)')
Pandas(Index=60389, user_id=666, item_id=707, rating=5, timestamp=880314103, title='Enchanted April (1991)')
Pandas(Index=92907, user_id=194, item_id=735, rating=4, timestamp=879524718, title='Philadelphia (1993)')
Pandas(Index=81945, user_id=178, item_id=157, rating=5, timestamp=882827400, title='Platoon (1986)')
Pandas(Index=42135, user_id=521, item_id=597, rating=2, timestamp=884476302, title='Eraser (1996)')
Pandas(Index=95363, user_id=671, item_id=1239, rating=3, timestamp=884036683, title='Cutthroat Island (1995)')
Pandas(Index=91591, user_id=804, item_id=925, rating=4, ti

Pandas(Index=66166, user_id=392, item_id=310, rating=4, timestamp=891037490, title='Rainmaker, The (1997)')
Pandas(Index=75202, user_id=795, item_id=1041, rating=3, timestamp=883254780, title='Forget Paris (1995)')
Pandas(Index=25991, user_id=545, item_id=79, rating=4, timestamp=879899233, title='Fugitive, The (1993)')
Pandas(Index=25114, user_id=244, item_id=164, rating=3, timestamp=880607154, title='Abyss, The (1989)')
Pandas(Index=35592, user_id=24, item_id=11, rating=5, timestamp=875323100, title='Seven (Se7en) (1995)')
Pandas(Index=71286, user_id=337, item_id=380, rating=4, timestamp=875185319, title='Star Trek: Generations (1994)')
Pandas(Index=33589, user_id=221, item_id=218, rating=4, timestamp=875246308, title='Cape Fear (1991)')
Pandas(Index=95186, user_id=715, item_id=1188, rating=2, timestamp=875964843, title='Young Guns II (1990)')
Pandas(Index=88236, user_id=368, item_id=844, rating=3, timestamp=889783453, title='Freeway (1996)')
Pandas(Index=64329, user_id=886, item_id=1

Pandas(Index=23715, user_id=454, item_id=117, rating=3, timestamp=888267343, title='Rock, The (1996)')
Pandas(Index=76153, user_id=99, item_id=363, rating=4, timestamp=885679262, title='Sudden Death (1995)')
Pandas(Index=98933, user_id=308, item_id=1252, rating=3, timestamp=887741604, title='Contempt (Mépris, Le) (1963)')
Pandas(Index=8647, user_id=792, item_id=100, rating=4, timestamp=877910822, title='Fargo (1996)')
Pandas(Index=82381, user_id=269, item_id=525, rating=4, timestamp=891449055, title='Big Sleep, The (1946)')
Pandas(Index=56426, user_id=487, item_id=81, rating=3, timestamp=883531507, title='Hudsucker Proxy, The (1994)')
Pandas(Index=25249, user_id=886, item_id=164, rating=4, timestamp=876033053, title='Abyss, The (1989)')
Pandas(Index=84535, user_id=456, item_id=697, rating=4, timestamp=881374390, title='Basketball Diaries, The (1995)')
Pandas(Index=82536, user_id=669, item_id=340, rating=4, timestamp=891182948, title='Boogie Nights (1997)')
Pandas(Index=26534, user_id=3

You can use the [pairwise_distances](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html) function from sklearn to calculate the cosine similarity. Note, the output will range from 0 to 1 since the ratings are all positive.

In [15]:
train_data_matrix

array([[5., 0., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.transpose(), metric='cosine')

In [25]:
user_similarity

array([[0.        , 0.85013599, 0.96186399, ..., 0.83964694, 0.72332613,
        0.8608991 ],
       [0.85013599, 0.        , 0.87860391, ..., 0.88695286, 0.90142989,
        0.86796859],
       [0.96186399, 0.87860391, 0.        , ..., 0.89988647, 0.97847875,
        1.        ],
       ...,
       [0.83964694, 0.88695286, 0.89988647, ..., 0.        , 0.8816065 ,
        0.78586086],
       [0.72332613, 0.90142989, 0.97847875, ..., 0.8816065 , 0.        ,
        0.86190055],
       [0.8608991 , 0.86796859, 1.        , ..., 0.78586086, 0.86190055,
        0.        ]])

Next step is to make predictions. You have already created similarity matrices: `user_similarity` and `item_similarity` and therefore you can make a prediction by applying following formula for user-based CF:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

You can look at the similarity between users *k* and *a* as weights that are multiplied by the ratings of a similar user *a* (corrected for the average rating of that user). You will need to normalize it so that the ratings stay between 1 and 5 and, as a final step, sum the average ratings for the user that you are trying to predict. 

The idea here is that some users may tend always to give high or low ratings to all movies. The relative difference in the ratings that these users give is more important than the absolute values. To give an example: suppose, user *k* gives 4 stars to his favourite movies and 3 stars to all other good movies. Suppose now that another user *t* rates movies that he/she likes with 5 stars, and the movies he/she fell asleep over with 3 stars. These two users could have a very similar taste but treat the rating system differently. 

When making a prediction for item-based CF you don't need to correct for users average rating since query user itself is used to do predictions.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [35]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [36]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [79]:
pd.DataFrame(item_prediction).iloc[0].sort_values(ascending = False).head()

1386    0.433258
1553    0.430569
1316    0.430468
1358    0.430468
1333    0.430468
Name: 0, dtype: float64

### Evaluation
There are many evaluation metrics but one of the most popular metric used to evaluate accuracy of predicted ratings is *Root Mean Squared Error (RMSE)*. 
<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

You can use the [mean_square_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) (MSE) function from `sklearn`, where the RMSE is just the square root of MSE. To read more about different evaluation metrics you can take a look at [this article](http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf). 

Since you only want to consider predicted ratings that are in the test dataset, you filter out all other elements in the prediction matrix with `prediction[ground_truth.nonzero()]`. 

In [81]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [86]:
user_prediction[test_data_matrix.nonzero()].flatten()

array([0.5623644 , 0.77892109, 0.86454599, ..., 0.17319561, 0.19251894,
       1.04285024])

In [88]:
test_data_matrix[test_data_matrix.nonzero()].flatten()

array([3., 3., 1., ..., 3., 4., 5.])

In [87]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.129756214058539
Item-based CF RMSE: 3.4555322268458992


Memory-based algorithms are easy to implement and produce reasonable prediction quality. 
The drawback of memory-based CF is that it doesn't scale to real-world scenarios and doesn't address the well-known cold-start problem, that is when new user or new item enters the system. Model-based CF methods are scalable and can deal with higher sparsity level than memory-based models, but also suffer when new users or items that don't have any ratings enter the system. I would like to thank Ethan Rosenthal for his [post](http://blog.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/) about Memory-Based Collaborative Filtering. 

# Model-based Collaborative Filtering

Model-based Collaborative Filtering is based on **matrix factorization (MF)** which has received greater exposure, mainly as an unsupervised learning method for latent variable decomposition and dimensionality reduction. Matrix factorization is widely used for recommender systems where it can deal better with scalability and sparsity than Memory-based CF. The goal of MF is to learn the latent preferences of users and the latent attributes of items from known ratings (learn features that describe the characteristics of ratings) to then predict the unknown ratings through the dot product of the latent features of users and items. 
When you have a very sparse matrix, with a lot of dimensions, by doing matrix factorization you can restructure the  user-item matrix into low-rank structure, and you can represent the matrix by the multiplication of two low-rank matrices, where the rows contain the latent vector. You fit this matrix to approximate your original matrix, as closely as possible, by multiplying the low-rank matrices together, which fills in the entries missing in the original matrix.

Let's calculate the sparsity level of MovieLens dataset:

In [89]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%')

The sparsity level of MovieLens100K is 93.7%


To give an example of the learned latent preferences of the users and items: let's say for the MovieLens dataset you have the following information: _(user id, age, location, gender, movie id, director, actor, language, year, rating)_. By applying matrix factorization the model learns that important user features are _age group (under 10, 10-18, 18-30, 30-90)_, _location_ and _gender_, and for movie features it learns that _decade_, _director_ and _actor_ are most important. Now if you look into the information you have stored, there is no such feature as the _decade_, but the model can learn on its own. The important aspect is that the CF model only uses data (user_id, movie_id, rating) to learn the latent features. If there is little data available model-based CF model will predict poorly, since it will be more difficult to learn the latent features. 

Models that use both ratings and content features are called **Hybrid Recommender Systems** where both Collaborative Filtering and Content-based Models are combined. Hybrid recommender systems usually show higher accuracy than Collaborative Filtering or Content-based Models on their own: they are capable to address the cold-start problem better since if you don't have any ratings for a user or an item you could use the metadata from the user or item to make a prediction. 

### SVD
A well-known matrix factorization method is **Singular value decomposition (SVD)**. Collaborative Filtering can be formulated by approximating a matrix `X` by using singular value decomposition. The winning team at the Netflix Prize competition used SVD matrix factorization models to produce product recommendations, for more information I recommend to read articles: [Netflix Recommendations: Beyond the 5 stars](http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html) and [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf).
The general equation can be expressed as follows:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />


Given `m x n` matrix `X`:
* *`U`* is an *`(m x r)`* orthogonal matrix
* *`S`* is an *`(r x r)`* diagonal matrix with non-negative real numbers on the diagonal
* *V^T* is an *`(r x n)`* orthogonal matrix

Elements on the diagnoal in `S` are known as *singular values of `X`*. 


Matrix *`X`* can be factorized to *`U`*, *`S`* and *`V`*. The *`U`* matrix represents the feature vectors corresponding to the users in the hidden feature space and the *`V`* matrix represents the feature vectors corresponding to the items in the hidden feature space.


Now you can make a prediction by taking dot product of *`U`*, *`S`* and *`V^T`*.



In [90]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF MSE: 2.718425917426652


Carelessly addressing only the relatively few known entries is highly prone to overfitting. SVD can be very slow and computationally expensive. More recent work minimizes the squared error by applying alternating least square or stochastic gradient descent and uses regularization terms to prevent overfitting. Alternating least square and stochastic gradient descent methods for CF will be covered in the next tutorials.


Review:

* We have covered how to implement simple **Collaborative Filtering** methods, both memory-based CF and model-based CF.
* **Memory-based models** are based on similarity between items or users, where we use cosine-similarity.
* **Model-based CF** is based on matrix factorization where we use SVD to factorize the matrix.
* Building recommender systems that perform well in cold-start scenarios (where little data is available on new users and items) remains a challenge. The standard collaborative filtering method performs poorly is such settings. 

## Looking for more?

If you want to tackle your own recommendation system analysis, check out these data sets. Note: The files are quite large in most cases, not all the links may stay up to host the data, but the majority of them still work. Or just Google for your own data set!

**Movies Recommendation:**

MovieLens - Movie Recommendation Data Sets http://www.grouplens.org/node/73

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Jester - Movie Ratings Data Sets (Collaborative Filtering Dataset) http://www.ieor.berkeley.edu/~goldberg/jester-data/

Cornell University - Movie-review data for use in sentiment-analysis experiments http://www.cs.cornell.edu/people/pabo/movie-review-data/

**Music Recommendation:**

Last.fm - Music Recommendation Data Sets http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/index.html

Yahoo! - Movie, Music, and Images Ratings Data Sets http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Audioscrobbler - Music Recommendation Data Sets http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html

Amazon - Audio CD recommendations http://131.193.40.52/data/

**Books Recommendation:**

Institut für Informatik, Universität Freiburg - Book Ratings Data Sets http://www.informatik.uni-freiburg.de/~cziegler/BX/
Food Recommendation:

Chicago Entree - Food Ratings Data Sets http://archive.ics.uci.edu/ml/datasets/Entree+Chicago+Recommendation+Data
Merchandise Recommendation:

**Healthcare Recommendation:**

Nursing Home - Provider Ratings Data Set http://data.medicare.gov/dataset/Nursing-Home-Compare-Provider-Ratings/mufm-vy8d

Hospital Ratings - Survey of Patients Hospital Experiences http://data.medicare.gov/dataset/Survey-of-Patients-Hospital-Experiences-HCAHPS-/rj76-22dk

**Dating Recommendation:**

www.libimseti.cz - Dating website recommendation (collaborative filtering) http://www.occamslab.com/petricek/data/
Scholarly Paper Recommendation:

National University of Singapore - Scholarly Paper Recommendation http://www.comp.nus.edu.sg/~sugiyama/SchPaperRecData.html

# Great Job!